In [2]:
import numpy as np

## Входные данные:

1. radius, length = длина канала
2. magma = начальная температура магмы (характеристика из монщности нагрева)
3. final_temparure = неиронично это финальная температура
4. alpha = коэф-т теплопроводности

In [4]:
radius, length = map(int, input().split())
magma = int(input())
final_temperature = 70
alpha = 0.56

print(radius, length, magma)

100 100 700


Будем двигаться по радиусу с шагом 10, а по времени замерим 10000 условных единиц времени

In [62]:
count_r = radius // 10
count_t = 10000

Сетка, по которой будем осуществлять вычисления

In [63]:
T = np.zeros((count_t, count_r))

Я не очень понимаю, как по мощности нагрева определить температуру воды у начала канала, поэтому решила просто взять какое-то нормальное распределение в районе температуры магмы (с отклонением 10). Я поискала формулы, там вроде как учитывается объем воды (единственное место, где пригодилась длина канала), возможно нужно будет ей воспользоваться

TODO: доделать

In [64]:
T[0, :] = np.random.normal(magma, 10, count_r)


In [65]:
T[0, 0] = T[0,0] * 0.7
T[0]

array([485.14428885, 694.67917623, 702.39768749, 707.23894235,
       694.5152844 , 710.65175205, 691.83186837, 708.0984112 ,
       705.43693901, 684.36842273])

Первоначальное распределение можно описать примерно так (да, я кажется перепутала индексы местами, но основная идея понятна):

![картинка из пэинта](sheme_0.png)


Теперь наше уравнение. По сетке оно выглядит вот так

$T_{i,j+1} = T_{ij} + \frac{\alpha \Delta t}{r_i \Delta r^2} (T_{i+1,j} - 2T_{ij} + T_{i-1, j}) $

где i = индекс по радиусу, j = индекс по времени.

То есть мы имеем i * j состояний. 

Так мы можем посчитать для всех внутренних точек, и для каждой внутренней точки перейти в следующее состояние T[i, 1]. Но на границах будет проблема: для них неизвестны i+1 или i-1. Я буду рассчитывать немного криво: например, зная для точки только i+1 значение, ему же буду приравнивать i-1. 

TODO: доделать (придумать что делать с крайними точками)

In [69]:
# Шаг по времени
dt = 100

In [70]:
T[1,0] = T[0, 0] + ((alpha * dt)/(10 * (10**2))) * (T[0, 1] - 2*T[0, 0] + T[0, 1])

T[1, 0]

508.61219623413575

Не остывает, поскольку как я понимаю, берет температуру у соседнего (так называемый *теплообмен*)Посмотрим что со всеми остальными

In [71]:
for i in range(1, count_r - 1):
    T[1, i] = T[0, i] + ((alpha * dt)/((i*count_r) * (count_r**2))) * (T[0, i+1] - 2*T[0, i] + T[0, i-1])


Границу считаю отдельно

In [73]:
T[1, -1] = T[0, -1] + ((alpha * dt)/((9*count_r) * (count_r**2))) * (T[0, 8] - 2*T[0, 9] + T[0, 8])

In [74]:
T[0]

array([485.14428885, 694.67917623, 702.39768749, 707.23894235,
       694.5152844 , 710.65175205, 691.83186837, 708.0984112 ,
       705.43693901, 684.36842273])

In [75]:
T[1]

array([508.61219623, 683.37745917, 702.31712431, 706.91106398,
       694.91932615, 710.26024091, 692.15934168, 707.94698708,
       705.3080897 , 684.63060871])

Получается, что где-то нагревается, а где-то остывает. Тут есть небольшой коэффициент, поясню:

0.97 = температура уменьшается на краю канала, поскольку она отдает тепло в землю


In [90]:
for j in range(2, count_t):
    T[j,0] = (T[j-1, 0] + ((alpha * dt)/(10 * (10**2))) * (T[j-1, 1] - 2*T[j-1, 0] + T[j-1, 1])) * 0.97
    for i in range(1, count_r - 1):
        T[j, i] = T[j-1, i] + ((alpha * dt)/((i*count_r) * (count_r**2))) * (T[j-1, i+1] - 2*T[j-1, i] + T[j-1, i-1])
    
    T[j, -1] = T[j-1, -1] + ((alpha * dt)/((9*count_r) * (count_r**2))) * (T[j-1, 8] - 2*T[j-1, 9] + T[j-1, 8])




In [91]:
T[2]

array([512.34032851, 674.65122569, 701.915444  , 706.60146467,
       695.30198329, 709.88569259, 692.47563477, 707.79957474,
       705.18181962, 684.88792847])

In [92]:
T[99]

array([398.24698997, 502.25511087, 592.50651671, 655.23185072,
       687.17557142, 698.43447239, 700.78711871, 701.99992969,
       700.16515505, 696.49289612])

In [93]:
T[9999]

array([ 58.35924958,  74.38915926,  90.2076525 , 105.51339949,
       119.91952878, 132.96239703, 144.11584767, 152.81180632,
       158.46769159, 160.52061356])

Получается что через 10000 условных единиц времени вода остыла и стала вот такой. Вообще, если заебываться, то можно взять еще одну сетку с разбиением 'длина канала' x 'радиус канала', измерять в каждом полученном кольце по такой схеме и передавать в следующее кольцо. Но мне кажется что уже и так выглядит хорошо. В общем, надо подумать, как это доделать